In [ ]:
!nvidia-smi

In [ ]:
!nvidia-smi

# Fresh bits of diffusers (needed for ModularPipeline/BriaFiboPipeline)
!pip -q install --upgrade "git+https://github.com/huggingface/diffusers" \
  transformers accelerate sentencepiece safetensors pillow boltons ujson

# Use PyTorch with CUDA 12.1 wheels in Colab
!pip -q install --upgrade --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio

# (Optional) If the model is gated on HF for you, uncomment to login interactively
# from huggingface_hub import notebook_login; notebook_login()


In [ ]:
import os, json, torch, platform
from PIL import Image
from diffusers import BriaFiboPipeline
from diffusers.modular_pipelines import ModularPipeline

print("torch:", torch.__version__, "cuda:", torch.cuda.is_available(), "cuda ver:", torch.version.cuda, "gpu:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)

# Pick dtype:
# - Colab T4 (sm_75) doesn't support bfloat16 well → use float16
# - A100 / L4 / H100 can do bfloat16 → use bfloat16
def pick_dtype():
    if not torch.cuda.is_available():
        return torch.float32
    major = torch.cuda.get_device_properties(0).major
    # Ampere (>=8) → bf16 ok, else fp16
    return torch.bfloat16 if major >= 8 else torch.float16

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = pick_dtype()
dtype


In [ ]:
!pip install --upgrade pip

# 1) Install a stable Pillow version that works with diffusers
!pip install --force-reinstall "pillow==11.0.0"

# 2) Install PyTorch with CUDA 12.1 (for Colab GPU)
!pip install --quiet --upgrade --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio

# 3) Install diffusers + friends (DO NOT reinstall pillow here)
!pip install --quiet --upgrade \
  "git+https://github.com/huggingface/diffusers" \
  transformers accelerate sentencepiece safetensors boltons ujson


In [ ]:
!pip install --upgrade pip

# 1) Pillow version that plays well with diffusers FIBO pipeline
!pip install --quiet "pillow==11.0.0"

# 2) diffusers (from GitHub) + friends
!pip install --quiet --upgrade \
  "git+https://github.com/huggingface/diffusers" \
  transformers accelerate sentencepiece safetensors boltons ujson


In [ ]:
import os, json, torch
from PIL import Image
from diffusers import BriaFiboPipeline
from diffusers.modular_pipelines import ModularPipeline

print("torch:", torch.__version__,
      "cuda_available:", torch.cuda.is_available(),
      "cuda_version:", torch.version.cuda,
      "pillow:", Image.__version__)

def pick_dtype():
    if not torch.cuda.is_available():
        return torch.float32
    major = torch.cuda.get_device_properties(0).major
    # Ampere and newer (A100 / L4 / H100, etc) → bfloat16 ok; else float16
    return torch.bfloat16 if major >= 8 else torch.float16

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = pick_dtype()
device, dtype


In [ ]:
!pip install -q huggingface_hub

from huggingface_hub import notebook_login
notebook_login()


In [ ]:
torch.set_grad_enabled(False)

# 1) Local VLM: NL → JSON
vlm_pipe = ModularPipeline.from_pretrained(
    "briaai/FIBO-VLM-prompt-to-JSON",
    trust_remote_code=True
)

# 2) FIBO generator
pipe = BriaFiboPipeline.from_pretrained(
    "briaai/FIBO",
    torch_dtype=dtype
).to(device)

def default_negative(existing_json: dict) -> str:
    if existing_json.get("style_medium", "").lower() in {"photograph", "photography", "photo"}:
        return "{'style_medium':'digital illustration','artistic_style':'non-realistic'}"
    return ""


In [ ]:
from IPython.display import display

out_gen = vlm_pipe(
    prompt="a studio product shot of a running shoe on matte black acrylic, soft rim light"
)
json_prompt_gen = out_gen.values["json_prompt"]

neg = default_negative(json.loads(json_prompt_gen))

generator = torch.Generator(device=device).manual_seed(12345)
res = pipe(
    prompt=json_prompt_gen,
    num_inference_steps=40,
    guidance_scale=5,
    negative_prompt=neg,
    generator=generator,
)
res.images[0].save("gen.png")
open("gen.json","w").write(json_prompt_gen)

display(Image.open("gen.png"))


In [ ]:
out_ref = vlm_pipe(
    json_prompt=json_prompt_gen,
    prompt="camera angle: low angle, lens: 85mm, rim light stronger"
)
json_prompt_ref = out_ref.values["json_prompt"]

res2 = pipe(
    prompt=json_prompt_ref,
    num_inference_steps=40,
    guidance_scale=5,
    negative_prompt=neg,
    generator=torch.Generator(device=device).manual_seed(12345),
)
res2.images[0].save("refine.png")
open("refine.json","w").write(json_prompt_ref)

display(Image.open("refine.png"))


In [ ]:
orig = Image.open("gen.png")
out_inspire = vlm_pipe(
    image=orig,
    prompt="make it futuristic with cool cyan accents"
)
json_prompt_insp = out_inspire.values["json_prompt"]

res3 = pipe(
    prompt=json_prompt_insp,
    num_inference_steps=40,
    guidance_scale=5,
    negative_prompt=neg,
    generator=torch.Generator(device=device).manual_seed(12345),
)
res3.images[0].save("inspire.png")
open("inspire.json","w").write(json_prompt_insp)

display(Image.open("inspire.png"))


In [ ]:
!pip install -q gradio


In [ ]:
import json
import torch
from PIL import Image
import gradio as gr
from diffusers import BriaFiboPipeline
from diffusers.modular_pipelines import ModularPipeline

# ---------- DEVICE & DTYPE ----------
def pick_dtype():
    if not torch.cuda.is_available():
        return torch.float32
    major = torch.cuda.get_device_properties(0).major
    return torch.bfloat16 if major >= 8 else torch.float16

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = pick_dtype()

torch.set_grad_enabled(False)

# ---------- LOAD MODELS (GLOBAL SINGLETONS) ----------
print("Loading VLM...")
vlm_pipe = ModularPipeline.from_pretrained(
    "briaai/FIBO-VLM-prompt-to-JSON",
    trust_remote_code=True
)

print("Loading FIBO generator...")
pipe = BriaFiboPipeline.from_pretrained(
    "briaai/FIBO",
    torch_dtype=dtype
).to(device)

print("Models loaded.")

# ---------- UTILITIES ----------
def default_negative(existing_json: dict) -> str:
    if existing_json.get("style_medium", "").lower() in {"photograph", "photography", "photo"}:
        return "{'style_medium':'digital illustration','artistic_style':'non-realistic'}"
    return ""

def apply_controls_to_json(base_json: dict,
                           camera_angle: str,
                           focal_length: int,
                           fov: int,
                           key_light_style: str,
                           palette: str):
    """Modify a parsed JSON prompt with UI controls."""
    j = dict(base_json)

    # Camera & lens
    if camera_angle:
        j["camera_angle"] = camera_angle
    j["lens_focal_length"] = f"{focal_length}mm"

    # We don't have true FOV in schema, but you can encode it as metadata / hint
    j.setdefault("camera", {})
    j["camera"]["field_of_view"] = f"{fov}deg"

    # Lighting
    if key_light_style:
        lighting = j.get("lighting", {})
        lighting["key_light"] = key_light_style
        j["lighting"] = lighting

    # Palette
    if palette:
        colors = [c.strip() for c in palette.split(",") if c.strip()]
        if colors:
            j["color_palette"] = colors

    return j

def run_fibo(json_obj: dict, seed: int = 12345, steps: int = 40, guidance: float = 5.0):
    json_str = json.dumps(json_obj)
    neg = default_negative(json_obj)

    gen = torch.Generator(device=device).manual_seed(seed)
    out = pipe(
        prompt=json_str,
        num_inference_steps=steps,
        guidance_scale=guidance,
        negative_prompt=neg,
        generator=gen
    )
    return out.images[0], json_str

# ---------- GRADIO HANDLERS ----------
def generate_from_text(prompt,
                       camera_angle,
                       focal_length,
                       fov,
                       key_light_style,
                       palette,
                       steps,
                       guidance,
                       seed):
    """Generate mode: NL → JSON → apply controls → image."""
    if not prompt:
        return None, None, "Please enter a prompt."

    # First: NL → JSON via VLM
    vlm_out = vlm_pipe(prompt=prompt)
    base_json_str = vlm_out.values["json_prompt"]
    base_json = json.loads(base_json_str)

    # Apply controls
    final_json = apply_controls_to_json(
        base_json,
        camera_angle=camera_angle,
        focal_length=focal_length,
        fov=fov,
        key_light_style=key_light_style,
        palette=palette,
    )

    img, final_json_str = run_fibo(final_json, seed=seed, steps=steps, guidance=guidance)
    pretty_json = json.dumps(final_json, indent=2, ensure_ascii=False)

    return img, pretty_json, ""


def refine_existing(json_in,
                    refine_instruction,
                    camera_angle,
                    focal_length,
                    fov,
                    key_light_style,
                    palette,
                    steps,
                    guidance,
                    seed):
    """Refine mode: existing JSON → optional VLM refine → apply controls → image."""
    if not json_in.strip():
        return None, None, "Paste a JSON prompt (generated from the left tab) to refine."

    try:
        current_json = json.loads(json_in)
    except Exception as e:
        return None, None, f"JSON parse error: {e}"

    refined = current_json
    if refine_instruction.strip():
        # Use VLM to refine the JSON but start from current_json
        vlm_out = vlm_pipe(json_prompt=json.dumps(current_json),
                           prompt=refine_instruction)
        refined = json.loads(vlm_out.values["json_prompt"])

    final_json = apply_controls_to_json(
        refined,
        camera_angle=camera_angle,
        focal_length=focal_length,
        fov=fov,
        key_light_style=key_light_style,
        palette=palette,
    )

    img, final_json_str = run_fibo(final_json, seed=seed, steps=steps, guidance=guidance)
    pretty_json = json.dumps(final_json, indent=2, ensure_ascii=False)
    return img, pretty_json, ""


def inspire_from_image(input_image,
                       inspire_instruction,
                       camera_angle,
                       focal_length,
                       fov,
                       key_light_style,
                       palette,
                       steps,
                       guidance,
                       seed):
    """Inspire mode: image → JSON → apply controls → image."""
    if input_image is None:
        return None, None, "Upload an image to inspire from."

    vlm_out = vlm_pipe(image=input_image, prompt=inspire_instruction or "")
    base_json = json.loads(vlm_out.values["json_prompt"])

    final_json = apply_controls_to_json(
        base_json,
        camera_angle=camera_angle,
        focal_length=focal_length,
        fov=fov,
        key_light_style=key_light_style,
        palette=palette,
    )

    img, final_json_str = run_fibo(final_json, seed=seed, steps=steps, guidance=guidance)
    pretty_json = json.dumps(final_json, indent=2, ensure_ascii=False)
    return img, pretty_json, ""


# ---------- GRADIO UI ----------
camera_angle_choices = [
    "",            # means "no override"
    "eye-level",
    "low",
    "high",
    "overhead",
    "dutch angle",
]

key_light_choices = [
    "",
    "soft rim light",
    "hard dramatic light",
    "soft studio key",
    "backlit silhouette"
]

with gr.Blocks(title="FIBO Studio - Camera & JSON Control") as demo:
    gr.Markdown(
        """
        # 🎥 FIBO Studio
        **JSON-native, controllable AI camera** built on Bria FIBO.
        - Type a prompt → see the structured JSON
        - Tweak camera, lens, FOV, lighting, palette
        - Regenerate with full reproducibility (seed)
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            with gr.Tab("Generate"):
                prompt = gr.Textbox(label="Natural language prompt", lines=3,
                                    value="a studio product shot of a running shoe on matte black acrylic, soft rim light")

                gen_camera_angle = gr.Dropdown(camera_angle_choices, label="Camera angle override", value="low")
                gen_focal = gr.Slider(18, 200, value=85, step=1, label="Lens focal length (mm)")
                gen_fov = gr.Slider(20, 120, value=60, step=1, label="Field of view (deg, hint)")
                gen_key_light = gr.Dropdown(key_light_choices, label="Key light style", value="soft rim light")
                gen_palette = gr.Textbox(label="Color palette (comma-separated)",
                                         value="black, cyan accents")

                gen_steps = gr.Slider(10, 60, value=40, step=1, label="Steps")
                gen_guidance = gr.Slider(1, 15, value=5, step=0.5, label="Guidance scale")
                gen_seed = gr.Number(value=12345, precision=0, label="Seed")

                gen_button = gr.Button("Generate")

            with gr.Tab("Refine"):
                refine_json_in = gr.Textbox(label="Existing JSON prompt", lines=10,
                                            placeholder="Paste JSON from the Generate tab to refine...")
                refine_instruction = gr.Textbox(
                    label="Refine instruction (e.g. 'make lighting more cinematic, stronger rim light')", lines=2)

                ref_camera_angle = gr.Dropdown(camera_angle_choices, label="Camera angle override", value="")
                ref_focal = gr.Slider(18, 200, value=85, step=1, label="Lens focal length (mm)")
                ref_fov = gr.Slider(20, 120, value=60, step=1, label="Field of view (deg, hint)")
                ref_key_light = gr.Dropdown(key_light_choices, label="Key light style", value="")
                ref_palette = gr.Textbox(label="Color palette (comma-separated)", value="")

                ref_steps = gr.Slider(10, 60, value=40, step=1, label="Steps")
                ref_guidance = gr.Slider(1, 15, value=5, step=0.5, label="Guidance scale")
                ref_seed = gr.Number(value=12345, precision=0, label="Seed")

                refine_button = gr.Button("Refine")

            with gr.Tab("Inspire"):
                inspire_image = gr.Image(type="pil", label="Inspiration image")
                inspire_instruction = gr.Textbox(
                    label="Inspire instruction (e.g. 'make it futuristic with cool cyan accents')", lines=2)

                insp_camera_angle = gr.Dropdown(camera_angle_choices, label="Camera angle override", value="")
                insp_focal = gr.Slider(18, 200, value=50, step=1, label="Lens focal length (mm)")
                insp_fov = gr.Slider(20, 120, value=60, step=1, label="Field of view (deg, hint)")
                insp_key_light = gr.Dropdown(key_light_choices, label="Key light style", value="")
                insp_palette = gr.Textbox(label="Color palette (comma-separated)", value="")

                insp_steps = gr.Slider(10, 60, value=40, step=1, label="Steps")
                insp_guidance = gr.Slider(1, 15, value=5, step=0.5, label="Guidance scale")
                insp_seed = gr.Number(value=12345, precision=0, label="Seed")

                inspire_button = gr.Button("Inspire")

        with gr.Column(scale=2):
            output_image = gr.Image(label="Output image")
            output_json = gr.Code(label="Final JSON prompt")
            status = gr.Markdown(label="Status / Messages")

    # --- Wire buttons ---
    gen_button.click(
        generate_from_text,
        inputs=[
            prompt,
            gen_camera_angle, gen_focal, gen_fov, gen_key_light, gen_palette,
            gen_steps, gen_guidance, gen_seed
        ],
        outputs=[output_image, output_json, status]
    )

    refine_button.click(
        refine_existing,
        inputs=[
            refine_json_in,
            refine_instruction,
            ref_camera_angle, ref_focal, ref_fov, ref_key_light, ref_palette,
            ref_steps, ref_guidance, ref_seed
        ],
        outputs=[output_image, output_json, status]
    )

    inspire_button.click(
        inspire_from_image,
        inputs=[
            inspire_image,
            inspire_instruction,
            insp_camera_angle, insp_focal, insp_fov, insp_key_light, insp_palette,
            insp_steps, insp_guidance, insp_seed
        ],
        outputs=[output_image, output_json, status]
    )

demo.launch(share=True)


In [ ]:
import json

comfy_workflow = {
    "nodes": [
        {
            "id": 1,
            "type": "LoadJSON",
            "title": "Structured JSON Prompt",
            "inputs": {},
            "outputs": {"json": "JSON"},
            "position": [50, 200]
        },
        {
            "id": 2,
            "type": "BriaFIBOFromJSON",
            "title": "FIBO Image Generator",
            "inputs": {
                "json_prompt": [1, "json"]
            },
            "outputs": {"image": "IMAGE"},
            "position": [350, 200]
        },
        {
            "id": 3,
            "type": "ColorGrade",
            "title": "Artist Color Grading",
            "inputs": {
                "image": [2, "image"]
            },
            "outputs": {"image": "IMAGE"},
            "position": [650, 120]
        },
        {
            "id": 4,
            "type": "Upscale",
            "title": "Upscale / Sharpen",
            "inputs": {
                "image": [3, "image"]
            },
            "outputs": {"image": "IMAGE"},
            "position": [950, 120]
        },
        {
            "id": 5,
            "type": "SaveImage",
            "title": "Final Output",
            "inputs": {
                "image": [4, "image"]
            },
            "outputs": {},
            "position": [1250, 120]
        }
    ]
}

with open("comfy_fibo_workflow.json", "w") as f:
    json.dump(comfy_workflow, f, indent=2)

print("ComfyUI workflow JSON saved.")


In [ ]:
!pip install graphviz


In [ ]:
from graphviz import Digraph

dot = Digraph(comment="FIBO ComfyUI Graph", format="png")
dot.attr(rankdir="LR", size="8,5")

dot.node("JSON", "Structured JSON\n(Camera • Lighting • Palette)", shape="box")
dot.node("FIBO", "Bria FIBO\n(JSON → Image)", shape="box", style="filled", fillcolor="lightblue")
dot.node("GRADE", "Color Grading\n(Artist Control)", shape="box")
dot.node("UPSCALE", "Upscale / Sharpen", shape="box")
dot.node("OUT", "Final Image", shape="box")

dot.edge("JSON", "FIBO")
dot.edge("FIBO", "GRADE")
dot.edge("GRADE", "UPSCALE")
dot.edge("UPSCALE", "OUT")

dot
